# Imports

In [1]:
import numpy as np
import uuid
import pandas as pd
import os
import bz2
import random
import scipy.stats as stats
import json
import seaborn as sns
import qwikidata
import matplotlib.pyplot as plt
from qwikidata.json_dump import WikidataJsonDump
#import pymongo
from qwikidata.linked_data_interface import get_entity_dict_from_api
from collections import Counter
from qwikidata.entity import WikidataItem, WikidataProperty, WikidataLexeme
import sqlite3
from itertools import islice

# Parse Dumps

In [2]:
#db_client = pymongo.MongoClient()
#mongo_db = db_client['wikidata_20200213']
#mongo_collection = mongo_db.claims_only

sql_db = sqlite3.connect('data/wikidata.db')
sql_cursor = sql_db.cursor()

wjd = WikidataJsonDump('data/latest-all.json.bz2') #Downloaded 13 Feb 2020

In [39]:
#sql_db.close()

In [7]:
sql_cursor.execute('''
    CREATE TABLE refs(
        entity_id TEXT,
        claim_id TEXT,
        claim_rank TEXT,
        property_id TEXT,
        reference_block_hash TEXT,
        reference_property_id TEXT,
        reference_index TEXT,
        reference_snaktype TEXT,
        reference_datatype TEXT,
        reference_value TEXT,
        PRIMARY KEY (
            entity_id,
            claim_id,
            claim_rank,
            property_id,
            reference_block_hash,
            reference_property_id,
            reference_index
        )
)''')
sql_db.commit()

OperationalError: near "references": syntax error

In [42]:
def consume(iterator, n=None):
    "Advance the iterator n-steps ahead. If n is None, consume entirely."
    # Use functions that consume iterators at C speed.
    if n is None:
        # feed the entire iterator into a zero-length deque
        collections.deque(iterator, maxlen=0)
    else:
        # advance to the empty slice starting at position n
        next(islice(iterator, n, n), None)

In [43]:
def get_reference_to_sqlite(e):
    for claim_group in e['claims']:
            for claim in e['claims'][claim_group]:
                if 'references' in claim:
                    for ref in claim['references']:
                        for ref_prop_id in ref['snaks-order']:
                            for j, snak in enumerate(ref['snaks'][ref_prop_id]):
                                if 'datavalue' in snak:
                                    value = (str(snak['datavalue']['value']))
                                else:
                                    value = 'none'
                                try:
                                    #print('Trying')
                                    sql_cursor.execute('''INSERT INTO refs(entity_id, property_id, claim_id, claim_rank,
                                    reference_block_hash, reference_property_id, reference_index, reference_snaktype, reference_datatype, reference_value)
                                    VALUES(?,?,?,?,?,?,?,?,?,?)''', (
                                        e['id'],
                                        claim['mainsnak']['property'],
                                        claim['id'],
                                        claim['rank'],
                                        ref['hash'],
                                        ref_prop_id,
                                        j,
                                        snak['snaktype'],
                                        snak['datatype'],
                                        value
                                    ))
                                except sqlite3.IntegrityError as err:
                                    sql_cursor.execute(
                                        '''SELECT reference_snaktype, reference_datatype, reference_value
                                        FROM refs 
                                        WHERE entity_id=? AND property_id=? AND claim_id=? AND claim_rank=? AND
                                        reference_block_hash=? AND reference_property_id=? AND reference_index=?''', (
                                        e['id'],
                                        claim['mainsnak']['property'],
                                        claim['id'],
                                        claim['rank'],
                                        ref['hash'],
                                        ref_prop_id,
                                        j
                                    ))
                                    conflicted_value = sql_cursor.fetchone()
                                    if conflicted_value[0] == snak['snaktype'] and conflicted_value[1] == snak['datatype'] and conflicted_value[2] == value:
                                        #print('Repeated')
                                        continue
                                    else:
                                        raise(err)
                else:
                    try:
                        sql_cursor.execute('''INSERT INTO refs(entity_id, property_id, claim_id, claim_rank,
                        reference_block_hash, reference_property_id, reference_index, reference_snaktype, reference_datatype, reference_value)
                        VALUES(?,?,?,?,?,?,?,?,?,?)''', (
                            e['id'],
                            claim['mainsnak']['property'],
                            claim['id'],
                            claim['rank'],
                            'none',
                            'none',
                            'none',
                            'none',
                            'none',
                            'none'
                        ))
                    except sqlite3.IntegrityError as err:
                        sql_cursor.execute(
                            '''SELECT reference_snaktype, reference_datatype, reference_value
                            FROM refs 
                            WHERE entity_id=? AND property_id=? AND claim_id=? AND claim_rank=? AND
                            reference_block_hash=? AND reference_property_id=? AND reference_index=?''', (
                            e['id'],
                            claim['mainsnak']['property'],
                            claim['id'],
                            claim['rank'],
                            'none',
                            'none',
                            'none'
                        ))
                        conflicted_value = sql_cursor.fetchone()
                        if conflicted_value[0] == 'none' and conflicted_value[1] == 'none' and conflicted_value[2] == 'none':
                            #print('Repeated')
                            continue
                        else:
                            raise(err)

In [44]:
np.random.seed(77175441)
picks = np.random.choice(77175441,size=int((.2)*77175441),replace=False)
picks = sorted(picks)

In [73]:
f = bz2.open('data/latest-all.json.bz2', mode="rb")
last_pick = -1

if last_pick == -1:
    savepoints = [None] * len(picks)
    index_start = 0
    f.seek(0)
    consume(f,1) #skipping first {
else:
    index_start = picks.index(last_pick)+1
    f.seek(savepoints[index_start-1])
    consume(f,1)  #skipping first {
try:
    for i, pick in enumerate(picks[index_start:]):
        consume(f, pick - last_pick - 1)
        savepoints[index_start+i] = f.tell()
        linebytes = f.readline()
        line_str = linebytes.decode('utf-8')
        line_str = line_str.rstrip(',\n')
        e = json.loads(line_str)
        #print(e['id']+20*' ')
        print(str(pick/77175441*100)+'%'+20*' ',end='\r')
        get_reference_to_sqlite(e)        
        last_pick = pick
        #print(index_start+i, pick, entity['id'], savepoints[index_start+i])
except Exception as err:
    print(err)
    print(e['id'])
finally:
    sql_db.commit()
    print('\n'+str(last_pick))
    #f.close()

Expecting value: line 1 column 1 (char 0)
Q8

76520571


TypeError: write() argument must be str, not list

In [ ]:
#j = []
#for i in range(len(savepoints)):
#    j.append({'pick':picks[i], 'savepoint':savepoints[i]})
with open('data/savepoints.json','w') as f:
    json.dump(j,f)

In [160]:
index_start = picks.index(last_pick)+1
picks[index_start:index_start+3], last_pick

([21, 29, 32], 16)

In [7]:
f = bz2.open('data/latest-all.json.bz2', mode="rb")
savepoint = 0
c = 0
f.seek(savepoint)
consume(f, picks[c])
try:
    for i, linebytes in enumerate(f):
        line_str = linebytes.decode("utf-8")
        line_str = line_str.rstrip(",\n")
        if line_str in ["[", "]"]:
            continue
        print(i-1,json.loads(line_str)['id'])
        c += 1
        print(str(c/77175441*100)+'%'+20*' ',end='\r')
        get_reference_to_sqlite(e)
        if c == len(picks):
            break
        else:
            consume(it, picks[c] - picks[c-1] - 1)
except Exception as err:
    print(err)
    print(e['id'],claim['mainsnak']['property'],claim['id'],claim['rank'],ref['hash'],ref_prop_id,j,snak['snaktype'],snak['datatype'],str(snak['datavalue']['value']))
finally:
    sql_db.commit()
savepoint = f.tell()
print('\n'+str(savepoint)+' ; '+str(c))
f.close()

KeyboardInterrupt: 